In [2]:
import tensorflow as tf
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts
import numpy as np

In [8]:
# -*- coding: utf-8 -*-
"""
Created on Sat Apr 25 23:56:44 2020

@author: Eddie
"""

#ToDo: Create a class that holds the board and bench for every player
#Shop has been reverted to all champs to test the DQN implementation

import copy
import random
# from ChampionLoader import ChampionLoader

class Agent(object):
    def __init__(self, name, gold, champion_loader = ChampionLoader()):
        self.board = {### TO DO
        }
        self.bench = {'c1': 'Empty',
                'c2': 'Empty',
                'c3': 'Empty',
                'c4': 'Empty',
                'c5': 'Empty',
                'c6': 'Empty',
                'c7': 'Empty',
                'c8': 'Empty',
                'c9': 'Empty'
        }
        self.gold = gold
        self.champion_loader = champion_loader
        self.shop = list(self.champion_loader.data.keys())
#         self.shop = random.sample(list(self.champion_loader.data.keys()), 5)
        self.hp = 100
        
    ### because we haven't implemented the full game mechanics yet, to test the DQN the mechanic will be only 1 champion at a time    
    def buyChampion(self, champion_to_buy):
        ### Check if the bench is full
        if "Empty" not in self.bench.values():
            return 1
        ### Check if shop has the champion
        elif champion_to_buy not in self.shop:
            return 2
        ### Check if player has enough gold
        elif self.champion_loader.getChampion(champion_to_buy)['cost'] > self.gold:
            return 3
        else:
            ### temporary
            self.bench['c1'] = champion_to_buy
#             position_for_champion = next((x for x in self.bench if self.bench[x] == 'Empty'))
#             self.bench[position_for_champion] = champion_to_buy
#             self.shop.remove(champion_to_buy)
            self.gold = self.gold - self.champion_loader.getChampion(champion_to_buy)['cost']

    def getStateCopy(self):
        return copy.deepcopy(self.state)

    def getAction(self, gameState):
        pass

    ### auto parameter allows for a shop refresh without costing gold
    def shopRefresh(self, auto=False):
        if self.gold < 2:
            return False
        elif auto:
            self.shop = list(self.champion_loader.data.keys())
#             self.shop = random.sample(list(self.champion_loader.data.keys()), 5)
#             self.gold = self.gold - 2
            return True
        else:
            self.shop = list(self.champion_loader.data.keys())
#             self.shop = random.sample(list(self.champion_loader.data.keys()), 5)
            self.gold = self.gold - 2
            return True
        
    def benchRefresh(self):
            self.bench = {'c1': 'Empty',
            'c2': 'Empty',
            'c3': 'Empty',
            'c4': 'Empty',
            'c5': 'Empty',
            'c6': 'Empty',
            'c7': 'Empty',
            'c8': 'Empty',
            'c9': 'Empty'
        }

In [5]:
# -*- coding: utf-8 -*-
"""
Created on Sat Apr 25 22:21:26 2020

@author: Eddie
"""

import pandas as pd
import collections

class ChampionLoader(object):
    def __init__(self):
        self.df = pd.read_csv('./data/champion_stats.csv')
        self.data = collections.defaultdict(dict)
        self.data_idx = collections.defaultdict(dict)
        for idx, r in self.df.iterrows():
            id = r['champion']
            self.data[id] = {
                'id': idx,
                'champion': r['champion'],
                'origin': r['origin'],
                'class': r['class'],
                'cost': None if pd.isnull(r['cost']) else int(r['cost']),
                'hp1': None if pd.isnull(r['hp1']) else int(r['hp1']),
                'hp2': None if pd.isnull(r['hp2']) else int(r['hp2']),
                'hp3': None if pd.isnull(r['hp3']) else int(r['hp3']),
                'dps1': None if pd.isnull(r['dps1']) else int(r['dps1']),
                'dps2': None if pd.isnull(r['dps2']) else int(r['dps2']),
                'dps3': None if pd.isnull(r['dps3']) else int(r['dps3']),
                'attack_speed': None if pd.isnull(r['attack_speed']) else float(r['attack_speed']),
                'damage1': None if pd.isnull(r['damage1']) else int(r['damage1']),
                'damage2': None if pd.isnull(r['damage2']) else int(r['damage2']),
                'damage3': None if pd.isnull(r['damage3']) else int(r['damage3']),
                'range': None if pd.isnull(r['range']) else int(r['range']),
                'armor': None if pd.isnull(r['armor']) else int(r['armor']),
                'magic_resist': None if pd.isnull(r['magic_resist']) else int(r['magic_resist'])
            }
        
        for idx, r in self.df.iterrows():
            self.data_idx[idx] = {'champion': r['champion']}
            
        del self.df

    def getChampion(self, champion):
        return self.data[champion]
    
    def getChampion_idx(self, idx):
        return self.data_idx[idx]
    

In [6]:
# -*- coding: utf-8 -*-
"""
Created on Sat Apr 25 23:36:36 2020

@author: Eddie
"""
import math

class Combat(object):
    def __init__(self, champion_loader, playeragent, opponentagent):
        self.champion_loader = champion_loader
        self.player_champion = playeragent.bench['c1']
        self.opponent_champion = opponentagent.bench['c1']
    
    def combatTurn(self, ms):        
        if (ms % math.floor(1000/self.player_champion['attack_speed'])) == 0:
            self.update_champions(self.player_champion, self.opponent_champion)
            print('Player champion attacks')
            print(ms)
            print(self.player_champion)
            print(self.opponent_champion)
        # Get champions to fight        
        if (ms % math.floor(1000/self.opponent_champion['attack_speed'])) == 0:
            self.update_champions(self.opponent_champion, self.player_champion)
            print('Opponent champion attacks')
            print(ms)
            print(self.player_champion)
            print(self.opponent_champion)

    def combatPrep(self, player_champion, player_champion_star, opponent_champion, opponent_champion_star):
        player_champion['current_hp'] = player_champion['hp'+player_champion_star]
        player_champion['current_damage'] = player_champion['damage'+player_champion_star]
        opponent_champion['current_hp'] = opponent_champion['hp'+opponent_champion_star]
        opponent_champion['current_damage'] = opponent_champion['damage'+opponent_champion_star]
        
    def combatEnd(self, ):
        if self.player_champion['current_hp'] == 0:
            return 'opponent'
        if self.opponent_champion['current_hp'] == 0:
            return 'player' 
        return False 
        
    def update_champions(self, attacker, defender):
        damage_multiplier = 100/(100+defender['armor'])
        damage = attacker['current_damage']*damage_multiplier
        defender['current_hp'] = max(defender['current_hp'] - damage, 0)

    def run(self):
        #ms stands for milleseconds because actions happen 
        ms = 0
        self.player_champion = self.champion_loader.getChampion(self.player_champion)
        self.opponent_champion = self.champion_loader.getChampion(self.opponent_champion)
        self.combatPrep(self.player_champion, '1', self.opponent_champion, '1')

        while not self.combatEnd() and ms < 50000:
            self.combatTurn(ms)
            ms += 1
        if not self.combatEnd():
            return 'timeout'
        else:
            return self.combatEnd()

In [25]:
# -*- coding: utf-8 -*-
"""
Created on Sat Apr 25 23:36:36 2020

@author: Eddie
"""

from Combat import Combat

class TFTGameEnv(py_environment.PyEnvironment):
    def __init__(self, playeragent, opponentagent, championloader):
        
        ### the action space here includes every champ, purchase XP, re-roll, and end turn
        self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int32, minimum=0, maximum=96, name='action')

        ### the observation space here includes all the champs in the store as well as all owned champs
        self._observation_spec = array_spec.BoundedArraySpec(shape=(1,), dtype=np.int32, minimum=0, name='observation')
        
        ### this is the reward
        self.reward = 0
        
        ### this is the episode-end and game-end check
        self._game_ended = False
        self._turn_ended = False
        
        ### these are the agents that are used
        self.playeragent = playeragent
        self.opponentagent = opponentagent
        
        ### this is the champion loader
        self.championloader = championloader
        
    def action_spec(self):
        return self._action_spec
    def observation_spec(self):
        return self._observation_spec
    
    ### ### Reset the entire game. Set the HP to 100, bench to empty, refresh shop, reset gold
    def _reset(self):
        self.playeragent.gold = 100
        self.playeragent.shopRefresh(auto=True)
        self.playeragent.benchRefresh()
        self.playeragent.hp = 100
        self.opponentagent.gold = 100
        self.opponentagent.shopRefresh(auto=True)
#         self.opponentagent.benchRefresh()
        self.opponentagent.hp = 100
        self.reward = 0
        self._game_ended = False
        return ts.restart(np.array([self.reward], dtype=np.int32))
    
    def _reward(self):
        ### this way, the lower the opponent hp, the higher the reward
        return self.playeragent.gold + (100 - self.opponentagent.hp) + self.playeragent.hp

    def _take_action(self, action):
        ### ### The next statements take in to account every action outcome.
        ### 0 means end turn, and if hp is 0 at the end then end game. loser loses 10 hp.
        ### 1 is re-roll
        ### 2 is purchase XP
        ### 3+ is purchase champions
        if action == 0:
            self._turn_ended = True
            combat_instance = Combat(self.championloader, self.playeragent, self.opponentagent)
            winner = combat_instance.run()
            if winner == 'opponent':
                self.playeragent.hp = self.playeragent.hp - 10
            elif winner == 'player':
                self.opponentagent.hp = self.opponentagent.hp - 10
            if self.playeragent.hp == 0 or self.opponentagent.hp == 0:
                self._game_ended = True

        elif action == 1:
            self.playeragent.shopRefresh(auto=False)          
        ### TO- DO:elif action == 2:
            ### TO-DO gain XP
            ### for now, so the code works, I have this as also a shop refresh
        elif action == 2:
            self.playeragent.shopRefresh(auto=False)
        elif action > 2 and action <= 96:
            champion_to_buy = self.championloader.getChampion_idx(action-3)['champion'] ### -3 here to account for actions 0 1 2
            print(champion_to_buy)
            self.playeragent.buyChampion(champion_to_buy)
        else:
            raise ValueError('Action not recognized')

    
    def _step(self, action):
        ### take the action
        self._take_action(action)
        
        if self._game_ended:
            # The last action ended the episode. Ignore the current action and start
            # a new episode.
            return self.reset()
        
        reward = self._reward()
        
        ### ### This is the calculation of the reward after the turn ends.
        if self._turn_ended:
            return ts.termination(np.array([self.reward], dtype=np.int32), reward)
        else:
            return ts.transition(np.array([self.reward], dtype=np.int32), reward=0.0, discount=1.0)

In [26]:
buy_champs_action = np.array([6,18,23], dtype=np.int32)
end_turn_action = np.array(0, dtype=np.int32)
playerAgent = Agent('Eddie', 100)
opponentAgent = Agent('Bob', 100)
opponentAgent.buyChampion('Fiora')
print(opponentAgent.bench)
Champ_Object = ChampionLoader()

{'c1': 'Fiora', 'c2': 'Empty', 'c3': 'Empty', 'c4': 'Empty', 'c5': 'Empty', 'c6': 'Empty', 'c7': 'Empty', 'c8': 'Empty', 'c9': 'Empty'}


In [27]:
environment = TFTGameEnv(playerAgent, opponentAgent, Champ_Object)

### Create training and evaluation python environments
train_py_env = TFTGameEnv(playerAgent, opponentAgent, Champ_Object)
eval_py_env = TFTGameEnv(playerAgent, opponentAgent, Champ_Object)

### Convert these environments to tensors
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

In [28]:
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 1000  # @param {type:"integer"} 
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

In [29]:
fc_layer_params = (100,)

q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)

In [30]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

In [31]:
eval_policy = agent.policy
collect_policy = agent.collect_policy
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(), train_env.action_spec())

In [32]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):
    total_return = 0.0
    counter = 0
    for _ in range(num_episodes):
        time_step = environment.reset()
        episode_return = 0.0

        while not time_step.is_last():
            action_step = policy.action(time_step)
            print(action_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward
            counter += 1
        print(counter)
        total_return += episode_return

    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]

compute_avg_return(eval_env, random_policy, num_eval_episodes)

PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([18])>, state=(), info=())
Dr-Mundo
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([3])>, state=(), info=())
Aatrox
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([20])>, state=(), info=())
Elise
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([35])>, state=(), info=())
Katarina
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([53])>, state=(), info=())
Mordekaiser
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([50])>, state=(), info=())
Maokai
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([39])>, state=(), info=())
Kindred
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([95])>, state=(), info=())
Zed
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([66])>, state=(), info=())
Qiyana
PolicyStep(action=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([89])>, state=(), i

189.0

In [58]:
time_step = environment.reset()
opponentAgent.buyChampion('Aatrox')
print(time_step)
cumulative_reward = time_step.reward
print(cumulative_reward)

TimeStep(step_type=array(0), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([0]))
0.0


In [59]:
print(playerAgent.gold)
for i in buy_champs_action:
    print(i)
    time_step = environment.step(i)
    print(playerAgent.bench)
    print(playerAgent.gold)
    print(time_step)
    cumulative_reward += time_step.reward

100
6
Anivia
{'c1': 'Anivia', 'c2': 'Empty', 'c3': 'Empty', 'c4': 'Empty', 'c5': 'Empty', 'c6': 'Empty', 'c7': 'Empty', 'c8': 'Empty', 'c9': 'Empty'}
95
TimeStep(step_type=array(1), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([0]))
18
Dr-Mundo
{'c1': 'Dr-Mundo', 'c2': 'Empty', 'c3': 'Empty', 'c4': 'Empty', 'c5': 'Empty', 'c6': 'Empty', 'c7': 'Empty', 'c8': 'Empty', 'c9': 'Empty'}
92
TimeStep(step_type=array(1), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([0]))
23
Fiora
{'c1': 'Fiora', 'c2': 'Empty', 'c3': 'Empty', 'c4': 'Empty', 'c5': 'Empty', 'c6': 'Empty', 'c7': 'Empty', 'c8': 'Empty', 'c9': 'Empty'}
91
TimeStep(step_type=array(1), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([0]))


In [60]:
time_step = environment.step(end_turn_action)

Player champion attacks
0
{'id': 20, 'champion': 'Fiora', 'origin': 'Noble', 'class': nan, 'cost': 1, 'hp1': 400, 'hp2': 720, 'hp3': 1440, 'dps1': 28, 'dps2': 50, 'dps3': 101, 'attack_speed': 1.0, 'damage1': 40, 'damage2': 72, 'damage3': 144, 'range': 1, 'armor': 25, 'magic_resist': 20, 'current_hp': 400, 'current_damage': 40}
{'id': 0, 'champion': 'Aatrox', 'origin': 'Light', 'class': 'Blademaster', 'cost': 3, 'hp1': 700, 'hp2': 1260, 'hp3': 2520, 'dps1': 42, 'dps2': 76, 'dps3': 152, 'attack_speed': 0.7, 'damage1': 65, 'damage2': 117, 'damage3': 234, 'range': 1, 'armor': 35, 'magic_resist': 20, 'current_hp': 670.3703703703703, 'current_damage': 65}
Opponent champion attacks
0
{'id': 20, 'champion': 'Fiora', 'origin': 'Noble', 'class': nan, 'cost': 1, 'hp1': 400, 'hp2': 720, 'hp3': 1440, 'dps1': 28, 'dps2': 50, 'dps3': 101, 'attack_speed': 1.0, 'damage1': 40, 'damage2': 72, 'damage3': 144, 'range': 1, 'armor': 25, 'magic_resist': 20, 'current_hp': 348.0, 'current_damage': 40}
{'id': 0,

In [61]:
print(time_step)
print(playerAgent.hp)
print(opponentAgent.hp)
print('Final Reward = ', time_step.reward)
print(playerAgent.gold)
print(playerAgent.bench)

TimeStep(step_type=array(2), reward=array(181., dtype=float32), discount=array(0., dtype=float32), observation=array([0]))
90
100
Final Reward =  181.0
91
{'c1': 'Fiora', 'c2': 'Empty', 'c3': 'Empty', 'c4': 'Empty', 'c5': 'Empty', 'c6': 'Empty', 'c7': 'Empty', 'c8': 'Empty', 'c9': 'Empty'}
